In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
data_train = pd.read_csv('train.csv')

### DATA CLEANING EDA


In [ ]:
data_train.head()

In [ ]:
data_train.info()

In [ ]:
data_train.describe()

In [ ]:
data_train.head()

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train['keyword'] = data_train['keyword'].fillna('none')

In [ ]:
data_train['location'] = data_train['location'].fillna('unknown')

In [ ]:
data_train.isnull().sum()

In [ ]:
def clean_text(text):
    
    text = re.sub(r'http\S+|www\.\S+', '', text)
   
    text = re.sub(r'\#\w+|@\w+', '', text)
   
    text = re.sub(r'[^a-zA-Z\s]', '', text)
  
    text = text.lower()
    return text

# Apply cleaning
data_train['cleaned_text'] = data_train['text'].apply(clean_text)

In [ ]:
data_train['cleaned_text']

In [ ]:
data_train[data_train['keyword'].isna()]

In [ ]:
data_train[data_train['location'].isna()]

In [ ]:


# Plot label distribution
sns.countplot(x='target', data=data_train)  # Replace 'label_column' with your label column
plt.title('Label Distribution')
plt.show()

In [ ]:
data_train['text_length'] = data_train['cleaned_text'].apply(len)

# Plot text length distribution
sns.histplot(data_train['text_length'], bins=30, kde=True)
plt.title('Text Length Distribution')
plt.show()
